# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 15](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=15)*

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-25"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse

import pandas as pd


# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    print(
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....'
    )
    print('TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.')
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
import lsst.ts.logging_and_reporting.almanac as alm
import lsst.ts.logging_and_reporting.reports as rep
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_sources import uniform_field_counts
from lsst.ts.logging_and_reporting.all_reports import AllReports

import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist

try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False

In [3]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
experimental = dict(
    exposure_type_tally=True,
    parse_error_in_log=True,
    )

In [4]:
# Set default env to "usdf" and try before PUSH to repo.
summit = "https://summit-lsp.lsst.codes"
usdf = "https://usdf-rsp-dev.slac.stanford.edu"
tucson = "https://tucson-teststand.lsst.codes"

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", summit
)  # TODO try with "usdf" before push (else "summit")

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(server_url=server,
                    min_dayobs=min_day_obs,
                    max_dayobs=max_day_obs,
                   )
allrep = AllReports(allsrc=allsrc)

<head>
<style>
code {
  font-family: Consolas,"courier new";
  color: crimson;
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 105%;
}
</style>
</head>

In [6]:
instrum_str = ', '.join(list(allsrc.exp_src.instruments.keys()))
md(f'# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}')

# Showing data for 2024-09-25 to 2024-09-26 for LSSTComCamSim, LSSTComCam, LATISS

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#almanac)
* [Summary plots of whole night](#Summary-Plots)
* [Summary Scalars for the night](#Summary-Scalars)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Exposure Log](#Exposure-Log)
    - AuxTel
    - Simonyi
* [Narrative Log](#Narrative-Log)
* [Developer Only](#dev-only)

## Night Report 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```
Daily checkouts, started at ~10:30 CLT and went well.

UPS failed, it was not charging and ran out of battery a couple of times. We follow AuxTel Recovery after Shutdown procedure, missed one system so checkout failed, issue fixed and checkout passed.

We started venting at ~16:20CLT.

Calibrations skipped.

Observations started 20:37 CLT and finished 01:30 CLT. No issues to report.
```

Telescope Status: ATTCS and LATISS in STANDBY.
Dome and Telescope at park position.

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

### Telescope: Simonyi

...

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

```
Commercial power was back at 15:25. We started the day plans for BLOCK-T151 Simonyi Hexapods Startup around 18:33 pm UTC. Then we warmed up hexapods (BLOCK-T3 and -T4) without an issue. Right after warming up we put hexapods back DISABLED and ran the MTRotator startup block (~19:30 PM UTC). MTRotator start-up, M1M3 opening and closing exercise were proceeded before lunch without any critical issue. 

After dinner, we changed MTPtg configuration to proceed Soak Test w/ Rotator and mtmount-ccw-only test (BLOCK-T170). The Scheduler1 was cycled several times to get some changes from Tiago R. There was no overcurrent (>3.5A) occurring during the soak test. 

Around 1:15am, we stopped scheduler and changed the MTPtg setup to monitor both MTRotator and MTMount as requested on the testcase. Both hexapods and rotator were all move to the origin and set to Standby. PDUs were off (BLOCK-T192 and BLOCK-T162).
```

Telescope Status: Simonyi telescope is parked at the Zenith. MTHexapod1, MTHexapod2, and MTRotator are on standby, and PDUs are off. MTPtg is set to monitor both MTRotator and MTMount. 

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

-------------

<a id="almanac"></a>
## Almanac

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-09-25**

,(times in UTC),
Moon Rise,2024-09-25 06:22:17.988,
Moon Set,2024-09-25 16:08:11.923,
Moon Illumination,46%,(% illuminated)
Astronomical Twilight (morning),2024-09-25 09:07:53.478,(-18 degrees)
Astronomical Twilight (evening),2024-09-25 00:00:53.995,(-18 degrees)
Solar Midnight,2024-09-25 04:34:25.909,
Nautical Twilight (morning),2024-09-25 09:36:06.914,(-12 degrees)
Nautical Twilight (evening),2024-09-24 23:32:39.307,(-12 degrees)
Civil Twilight (morning),2024-09-25 10:04:03.933,(-6 degrees)
Civil Twilight (evening),2024-09-24 23:04:40.798,(-6 degrees)


<a id="Summary-Plots"></a>
## Summary plots of whole night 

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps 
#rollup,detail = allsrc.get_observation_gaps()
#allrep.plot_observation_gap_detail(detail)

In [10]:
# Exposure Tally
md('<a id="Summary-Scalars"></a>')
md(f'## Summary Scalars for the night ')
md('(of all available instruments that have at least one exposure)')
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

<a id="Summary-Scalars"></a>

## Summary Scalars for the night 

(of all available instruments that have at least one exposure)

,LATISS
Total Night (HH:MM:SS),9:06:59
Total Exposure (HH:MM:SS),2:38:31
Number of exposures,252
Number of slews,46
Total Detector Read (HH:MM:SS),0:09:39
Mean Detector Read (HH:MM:SS),0:00:02
Total Slew (HH:MM:SS),0:33:45
Mean Slew (HH:MM:SS),0:00:44
Total Idle (HH:MM:SS),5:54:41


## Jira Tickets (Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {rep.mdpathlink(url)}")

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status['reports']['endpoint_url']
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://summit-lsp.lsst.codes/nightreport/reports?is_human=either&is_valid=true&min_day_obs=20240925&max_day_obs=20240926&limit=999) in `confluence_url`.

## BLOCKS observed
(TODO: BLOCKS observed) 

## Exposure Log

### EXPERIMENTAL: Tally exposures with specific values of selected Fields
TODO: Remove this or replace with specific values that are deemed useful.

In [13]:
# Exposure Report
# Facet counts
if experimental.get('exposure_type_tally'):
    counts = uniform_field_counts(allsrc.exp_src.exposures['LATISS'])
    for fname, tally in counts.items():
        md(f'#### Tally of values for Field: {fname}')
        display(pd.DataFrame.from_dict(tally, orient='index'))

#### Tally of values for Field: observation_type

,0
bias,4
engtest,7
acq,65
cwfs,10
science,166


#### Tally of values for Field: observation_reason

,0
bias,4
engtest,7
acquisition,8
intra,5
extra,5
final_infocus,3
acq,54
object,54
x_offset_0,26
x_offset_-50,27


#### Tally of values for Field: target_name

,0
Park position,7
DaytimeCheckout001,2
DaytimeCheckout002,2
HD 175526,7
HD185975,21
HD160617,77
HD2811,83
HD 209036,7
HD205905,7
HD 214405,7


#### Tally of values for Field: science_program

,0
unknown,17
BLOCK-T17,2
BLOCK-305,13
spec-survey,188
BLOCK-306,32


| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

### Instrument: LATISS

* AT_O_20240925_000025 : 2024-09-26T00:50:10.749109
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000030 : 2024-09-26T00:50:11.797057
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000031 : 2024-09-26T00:50:11.846362
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000032 : 2024-09-26T00:50:11.896179
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

...

* AT_O_20240925_000059 : 2024-09-26T00:50:12.237075
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000065 : 2024-09-26T00:50:12.285017
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000066 : 2024-09-26T00:50:12.331740
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

* AT_O_20240925_000067 : 2024-09-26T00:51:03.066618
    - <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font>`High mount motion jitter`

-------------

## Narrative Log
(Now "linked things" from /narrativelog/messages.urls)

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-09-25 00:02:37.781499** <pre>Salindex 200793


Fail the 3er intent, this is a file system issue</pre>

- **2024-09-25 00:07:47.736800** <pre>Rotator motion through +/- 5 deg with motion confirmed through Chrono Dashboard and the CCW  dial in TMA EUI 




Rotator motion --&gt; 10 deg: SUCCESS
Rotator motion --&gt; -10 deg: SUCCESS
Rotator motion --&gt; 10 deg: SUCCESS
Rotator motion --&gt; -10 deg: SUCCESS


Because I forgot to move to 0 -- &gt;  SUCCESS
Rotator motion --&gt; 10 deg: SUCCESS
Rotator motion --&gt; -10 deg: SUCCESS</pre>

- **2024-09-25 00:09:06.357429** <pre>Rotator back to 0</pre>

- **2024-09-25 00:15:43.274896** <pre>Rotator motion through +/- 25 deg with motion confirmed through Chrono Dashboard and the CCW  dial in TMA EUI 


Rotator motion --&gt; 25 deg: SUCCESS
Rotator motion --&gt; -25 deg: SUCCESS</pre>

- **2024-09-25 00:17:04.991569** <pre>Rotator back at 0</pre>

...

- **2024-09-25 22:51:52.132772** <pre>Resume the running BLOCK-T170.</pre>

- **2024-09-25 23:02:07.968464** <pre>Tiago R. mentioned: I think there's an issue with the Scheduler configuration, I am investigating.. you can leave it running for now but I think we will need to restart it</pre>

- **2024-09-25 23:13:09.945523** <pre>AuxTel is ready to go on sky</pre>

- **2024-09-25 23:42:30.219957** <pre>Observations started</pre>

-------------

<a id="dev-only"></a>
# Developer Only Section 
Contains stuff only expected to be useful to developers.

May also contain sections that have moved out of he user section because they are not defined in the Storyboard.

## Experimental 

In [15]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

records = [dict(a=i, b=i, c=i, d=i, e=i) for i in range(100)]

def records_toggle(show_records):
    if show_records:
        return(pd.DataFrame(records))
    else:
        return print(f'Number of records: {len(records)}')
interact(records_toggle, show_records=False);

interactive(children=(Checkbox(value=False, description='show_records'), Output()), _dom_classes=('widget-inte…

In [16]:
def f(x):
    return x

interact(f, x=True);

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

## Overview 


In [16]:
# Display overview of Report context
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
"""
)

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://summit-lsp.lsst.codes/**
- Report **1 observing night(s)** with the last reported night starting on **2024-09-25 12:00:00**.
- min_dayobs='2024-09-25', max_dayobs='2024-09-26'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [17]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://summit-lsp.lsst.codes/rubintv-dev/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [4]

None


- Endpoint: https://summit-lsp.lsst.codes/nightreport/reports?is_human=either&is_valid=true&min_day_obs=20240925&max_day_obs=20240926&limit=999

None
- Got 4 records.  


### Overview for Service: `exposurelog` [14]

...

None
- Got 14 records.  


### Overview for Service: `narrativelog` [80]

None


- Endpoint: https://summit-lsp.lsst.codes/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_begin&min_date_added=2024-09-25T00%3A00%3A00&max_date_added=2024-09-26T00%3A00%3A00&limit=999

None
- Got 80 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [20]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-10-18 11:32:18.426770
